In [ ]:
%run ./utils.ipynb

In [ ]:
# Hyperparameters
mult = 1 # channel multiplier
folds_to_train = range(n_splits)

In [ ]:
sz = 202
max_crop_sz = int(sz * 0.9)

trn_tfms = albumentations.Compose([
    albumentations.HorizontalFlip(),
    albumentations.RandomSizedCrop((max_crop_sz, max_crop_sz), sz, sz, interpolation=1, p=0.5),
    albumentations.IAAAffine(rotate=10, p=0.5, mode='edge'),
    albumentations.Blur()
])

In [ ]:
base_tfms = albumentations.Compose([
    albumentations.PadIfNeeded(256, 256)
])

def get_data_bunch(sz=128, bs=64, part=0, trn_tfms=trn_tfms):
    trn_ds = FilesDataset(PATH/TRAIN_DN, sz, take_idxs=trn_parts[part], tfms=albumentations.Compose([trn_tfms, base_tfms]))
    val_ds = FilesDataset(PATH/TRAIN_DN, sz, take_idxs=val_parts[part], tfms=base_tfms)
    tst_ds = FilesDataset(PATH/TEST_DN, sz, tfms=base_tfms)
    
    trn_dl = DataLoader(trn_ds, bs, True, num_workers=11, pin_memory=True)
    val_dl = DataLoader(val_ds, bs, False, num_workers=11, pin_memory=True)
    tst_dl = DataLoader(tst_ds, bs, False, num_workers=11, pin_memory=True)
    
    trn_dl, val_dl, tst_dl = map(lambda ts: DeviceDataLoader(*ts), zip([trn_dl, val_dl, tst_dl], [default_device] * 3) )
    return DataBunch(trn_dl, val_dl, tst_dl, default_device)

def predict(model, dl, flip=True):
    model.eval()
    preds = [torch.sigmoid(model(xb)).detach().cpu().numpy()[:, 27:-27, 27:-27] for xb, yb in dl]
    preds = np.concatenate(preds)
    if not flip: return preds
    flipped_preds = np.concatenate([torch.sigmoid(model(torch.flip(xb, [3]))).detach().cpu().numpy()[:, 27:-27, 27:-27] for xb, yb in dl])
    preds = (preds + flipped_preds[:,:,::-1]) / 2
    return preds

def predict_with_TTA(model, dl, upside_down=True):
    model.eval()
    
    preds = np.concatenate([torch.sigmoid(model(xb)).detach().cpu().numpy()[:, 27:-27, 27:-27] for xb, yb in dl])
    if upside_down: preds_upside_down = np.concatenate([torch.sigmoid(model(torch.flip(xb, [2]))).detach().cpu().numpy()[:, 27:-27, 27:-27] for xb, yb in dl])
    
    flipped_preds = np.concatenate([torch.sigmoid(model(torch.flip(xb, [3]))).detach().cpu().numpy()[:, 27:-27, 27:-27] for xb, yb in dl])
    if upside_down: flipped_preds_upside_down = np.concatenate([torch.sigmoid(model(torch.flip(xb, [1, 3]))).detach().cpu().numpy()[:, 27:-27, 27:-27] for xb, yb in dl])
    
    preds = (preds + flipped_preds[:,:,::-1]) / 2
    if upside_down: preds = 0.5 * preds + (preds_upside_down[:,::-1,:] + flipped_preds_upside_down[:,::-1,::-1]) / 4 
    return preds

def predict_with_targs_and_TTA(model, dl, upside_down=True):
    preds = predict_with_TTA(model, dl, upside_down)
    targs = np.concatenate([yb.detach().cpu().numpy()[:, 27:-27, 27:-27] for xb, yb in dl])
    return preds, targs

def accuracy_no_pad(preds, targs):
    return accuracy_thresh(preds[:, 27:-27, 27:-27], targs[:, 27:-27, 27:-27])

def dice_no_pad(preds, targs):
    return dice(preds[:, 27:-27, 27:-27], targs[:, 27:-27, 27:-27])

def iou_pytorch(out, yb):
    preds = out > 0
    return torch.tensor(iou_metric(yb.cpu().numpy()[:, 27:-27, 27:-27], preds.cpu().numpy()[:, 27:-27, 27:-27]))

In [ ]:
class UnetBlock(nn.Module):
    def __init__(self, up_in, x_in, n_out, kernel_size=2, output_padding=0, padding=0, stride=2):
        super().__init__()
        up_out = x_out = n_out//2
        self.x_conv  = nn.Conv2d(x_in,  x_out,  1)
        self.tr_conv = nn.ConvTranspose2d(up_in, up_out, kernel_size, stride=stride, output_padding=output_padding, padding=padding)
        self.bn = nn.BatchNorm2d(n_out)
        self.out_channels = n_out
        
    def forward(self, up_p, x_p):
        up_p = self.tr_conv(up_p)
        x_p = self.x_conv(x_p)
        cat_p = torch.cat([up_p,x_p], dim=1)
        return self.bn(F.relu(cat_p))
    
class hcCombBlock(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.conv1 = nn.Conv2d(n_in, n_out // 2, 3, padding=1)
        self.conv2 = nn.Conv2d(n_out // 2, n_out, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(n_out // 2)
        self.bn2 = nn.BatchNorm2d(n_out)
    
    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.bn2(F.relu(self.conv2(x)))
        return x
        

class UnetIncV4(nn.Module):
    def __init__(self, inc):
        super().__init__()
        self.inc = inc
        self.sfs = [SaveFeatures(self.inc[0][i]) for i in [0,2,3,4,9,17,21]]
        self.up1 = UnetBlock(1536,1024,320 * mult,3,1)
        self.up2 = UnetBlock(320 * mult,384,160 * mult,kernel_size=3)
        self.up3 = UnetBlock(160  * mult,192,96 * mult,kernel_size=4)
        self.up4 = UnetBlock(96 * mult,160,128 * mult,kernel_size=3,stride=1)
        self.up5 = UnetBlock(128 * mult,64,48 * mult,kernel_size=3)
        self.up6 = UnetBlock(48 * mult,32,64 * mult,kernel_size=3,stride=1)
        self.up7 = UnetBlock(64 * mult,3,32 * mult,kernel_size=4)
        self.up8 = nn.ConvTranspose2d(32 * mult + 16, 1, 1)
        
        self.se1 = scSELayer(self.up1.out_channels, 8)
        self.se2 = scSELayer(self.up2.out_channels, 8)
        self.se3 = scSELayer(self.up3.out_channels, 8)
        self.se4 = scSELayer(self.up4.out_channels, 8)
        self.se5 = scSELayer(self.up5.out_channels, 8)
        self.se6 = scSELayer(self.up6.out_channels, 8)
        self.se7 = scSELayer(self.up7.out_channels, 8)
        
        self.se_feat0 = scSELayer(32)
        self.se_feat1 = scSELayer(64)
        self.se_feat2 = scSELayer(160)
        self.se_feat3 = scSELayer(192)
        self.se_feat4 = scSELayer(384)
        self.se_feat5 = scSELayer(1024)
        
        self.hc1 = HCBlock(self.up1.out_channels)
        self.hc2 = HCBlock(self.up2.out_channels)
        self.hc3 = HCBlock(self.up3.out_channels)
        self.hc4 = HCBlock(self.up4.out_channels)
        self.hc5 = HCBlock(self.up5.out_channels)
        self.hc6 = HCBlock(self.up6.out_channels)
                
        self.hc_comb = nn.Conv2d(96, 16, 3, padding=1)
        self.hc_bn = nn.BatchNorm2d(16)
        
    def forward(self,x):
        inp = x
        x = F.relu(self.inc(x))
        
        x = self.up1(x, self.se_feat5(self.sfs[5].features))
        x = self.se1(x)
        hc1 = self.hc1(x)
        
        x = self.up2(x, self.se_feat4(self.sfs[4].features))
        x = self.se2(x)
        hc2 = self.hc2(x)
        
        x = self.up3(x, self.se_feat3(self.sfs[3].features))
        x = self.se3(x)
        hc3 = self.hc3(x)
        
        x = self.up4(x, self.se_feat2(self.sfs[2].features))
        x = self.se4(x)
        hc4 = self.hc4(x)
        
        x = self.up5(x, self.se_feat1(self.sfs[1].features))
        x = self.se5(x)
        hc5 = self.hc5(x)
        
        x = self.up6(x, self.se_feat0(self.sfs[0].features))
        x = self.se6(x)
        hc6 = self.hc6(x)
    
        x = self.up7(x, inp)
        x = self.se7(x)
        
        hc = self.hc_comb(torch.cat((hc1, hc2, hc3, hc4, hc5, hc6), dim=1))
        hc = self.hc_bn(F.relu(hc))
        x = torch.cat((x, hc), dim=1)
        
        x = self.up8(x)
        return x[:,0]
    
    def close(self):
        for sf in self.sfs: sf.remove()

In [ ]:
def get_learner(db):
    incv4 = pretrainedmodels.inceptionv4()
    base = nn.Sequential(*list(incv4.children())[:-2])
    m = UnetIncV4(base)
    m.cuda(default_device)
    learn = Learner(db, m, true_wd=True, loss_fn=bce_loss, opt_fn=lambda x: optim.SGD(x))
#     learn = Learner(db, m, true_wd=True, loss_fn=bce_loss, opt_fn=AdamW)
    learn.metrics = [accuracy_no_pad, dice_no_pad, iou_pytorch]
    learn.callbacks = [SaveBest()]
    return learn

In [ ]:
name = 'incv4'

In [ ]:
%%time
for fold in folds_to_train:
    print(f'### Starting to train fold {fold} ###')
    db = get_data_bunch(sz=202, bs=16, part=fold, trn_tfms=trn_tfms)
    learn = get_learner(db)
    requires_grad(learn.model, True)
    requires_grad(learn.model.inc, False)
    learn.fit_one_cycle(9, 1e-1)
    learn.unfreeze()
    learn.fit_one_cycle(40, 1e-2)
    learn.loss_fn = lovasz_loss
    learn.fit_one_cycle(60, 5e-3)
    learn.save(f'{name}_fold{fold}')

In [ ]:
%%time

upside_down = False

for fold in folds_to_train:
    db = get_data_bunch(sz=202, bs=16, part=fold, trn_tfms=trn_tfms)
    learn = get_learner(db)
    
    learn.load(f'{name}_fold{fold}')
    val_preds, val_targs = predict_with_targs_and_TTA(learn.model, db.valid_dl, upside_down)
    test_preds = predict_with_TTA(learn.model, db.test_dl, upside_down)
    val_preds, test_preds = normalize_t(val_preds, val_targs, test_preds)
    print(f'Fold {fold} val acc: {accuracy_np(val_preds, val_targs)}, iou: {iou_metric(val_targs, val_preds > 0.5)}')
    
    np.save(f'/home/radek/db/salt/val_preds_{name}_fold{fold}', val_preds)
    np.save(f'/home/radek/db/salt/val_targs_{name}_fold{fold}', val_targs)
    np.save(f'/home/radek/db/salt/test_preds_{name}_fold{fold}', test_preds)
    del val_preds, val_targs, test_preds
    
    
    learn.load(f'{name}_best_iou_fold{fold}')
    val_preds, val_targs = predict_with_targs_and_TTA(learn.model, db.valid_dl, upside_down)
    test_preds = predict_with_TTA(learn.model, db.test_dl, upside_down)
    
    val_preds, test_preds = normalize_t(val_preds, val_targs, test_preds)
    print(f'Fold {fold} best iou val acc: {accuracy_np(val_preds, val_targs)}, iou: {iou_metric(val_targs, val_preds > 0.5)}')
    
    np.save(f'/home/radek/db/salt/val_preds_{name}_best_iou_fold{fold}', val_preds)
    np.save(f'/home/radek/db/salt/test_{name}_best_iou_fold{fold}', test_preds)
    del val_preds, val_targs, test_preds
    
    learn.model.close()
    del learn

In [ ]:
%%time

upside_down = False

for fold in folds_to_train:
    db = get_data_bunch(sz=202, bs=16, part=fold, trn_tfms=trn_tfms)
    learn = get_learner(db)
    
    learn.load(f'{name}_fold{fold}')
    val_preds, val_targs = predict_with_targs_and_TTA(learn.model, db.valid_dl, upside_down)
    test_preds = predict_with_TTA(learn.model, db.test_dl, upside_down)
    val_preds, test_preds = normalize_t(val_preds, val_targs, test_preds)
    print(f'Fold {fold} val acc: {accuracy_np(val_preds, val_targs)}, iou: {iou_metric(val_targs, val_preds > 0.5)}')
    
    np.save(f'/home/radek/db/salt/val_preds_{name}_fold{fold}', val_preds)
    np.save(f'/home/radek/db/salt/val_targs_{name}_fold{fold}', val_targs)
    np.save(f'/home/radek/db/salt/test_preds_{name}_fold{fold}', test_preds)
    del val_preds, val_targs, test_preds
    
    
    learn.load(f'{name}_best_iou_fold{fold}')
    val_preds, val_targs = predict_with_targs_and_TTA(learn.model, db.valid_dl, upside_down)
    test_preds = predict_with_TTA(learn.model, db.test_dl, upside_down)
    
    val_preds, test_preds = normalize_t(val_preds, val_targs, test_preds)
    print(f'Fold {fold} best iou val acc: {accuracy_np(val_preds, val_targs)}, iou: {iou_metric(val_targs, val_preds > 0.5)}')
    
    np.save(f'/home/radek/db/salt/val_preds_{name}_best_iou_fold{fold}', val_preds)
    np.save(f'/home/radek/db/salt/test_{name}_best_iou_fold{fold}', test_preds)
    del val_preds, val_targs, test_preds
    
    learn.model.close()
    del learn

In [ ]:
%%time

upside_down = False

for fold in [9]:
    db = get_data_bunch(sz=202, bs=16, part=fold, trn_tfms=trn_tfms)
    learn = get_learner(db)
    
    learn.load(f'{name}_fold{fold}')
    val_preds, val_targs = predict_with_targs_and_TTA(learn.model, db.valid_dl, upside_down)
    test_preds = predict_with_TTA(learn.model, db.test_dl, upside_down)
    val_preds, test_preds = normalize_t(val_preds, val_targs, test_preds)
    print(f'Fold {fold} val acc: {accuracy_np(val_preds, val_targs)}, iou: {iou_metric(val_targs, val_preds > 0.5)}')
    
    np.save(f'/home/radek/db/salt/val_preds_{name}_fold{fold}', val_preds)
    np.save(f'/home/radek/db/salt/val_targs_{name}_fold{fold}', val_targs)
    np.save(f'/home/radek/db/salt/test_preds_{name}_fold{fold}', test_preds)
    del val_preds, val_targs, test_preds
    
    
    learn.load(f'{name}_best_iou_fold{fold}')
    val_preds, val_targs = predict_with_targs_and_TTA(learn.model, db.valid_dl, upside_down)
    test_preds = predict_with_TTA(learn.model, db.test_dl, upside_down)
    
    val_preds, test_preds = normalize_t(val_preds, val_targs, test_preds)
    print(f'Fold {fold} best iou val acc: {accuracy_np(val_preds, val_targs)}, iou: {iou_metric(val_targs, val_preds > 0.5)}')
    
    np.save(f'/home/radek/db/salt/val_preds_{name}_best_iou_fold{fold}', val_preds)
    np.save(f'/home/radek/db/salt/test_{name}_best_iou_fold{fold}', test_preds)
    del val_preds, val_targs, test_preds
    
    learn.model.close()
    del learn

In [ ]:
iou_metric(val_targs, val_preds > 0.5)

In [ ]:
%%time

upside_down = False

for fold in folds_to_train:
    db = get_data_bunch(sz=202, bs=16, part=fold, trn_tfms=trn_tfms)
    learn = get_learner(db)
    
    learn.load(f'{name}_fold{fold}')
    val_preds, val_targs = predict_with_targs_and_TTA(learn.model, db.valid_dl, upside_down)
    test_preds = predict_with_TTA(learn.model, db.test_dl, upside_down)
    print(f'Fold {fold} val acc: {accuracy_np(val_preds, val_targs)}, iou: {iou_metric(val_targs, val_preds > 0.5)}')
    
    val_preds, test_preds = normalize_t(val_preds, val_targs, test_preds)
    
    np.save(f'/home/radek/db/salt/val_preds_{name}_fold{fold}', val_preds)
    np.save(f'/home/radek/db/salt/val_targs_{name}_fold{fold}', val_targs)
    np.save(f'/home/radek/db/salt/test_preds_{name}_fold{fold}', test_preds)
    del val_preds, val_targs, test_preds
    
    
    learn.load(f'{name}_best_iou_fold{fold}')
    val_preds, val_targs = predict_with_targs_and_TTA(learn.model, db.valid_dl, upside_down)
    test_preds = predict_with_TTA(learn.model, db.test_dl, upside_down)
    
    val_preds, test_preds = normalize_t(val_preds, val_targs, test_preds)
    print(f'Fold {fold} best iou val acc: {accuracy_np(val_preds, val_targs)}, iou: {iou_metric(val_targs, val_preds > 0.5)}')
    
    np.save(f'/home/radek/db/salt/val_preds_{name}_best_iou_fold{fold}', val_preds)
    np.save(f'/home/radek/db/salt/test_{name}_best_iou_fold{fold}', test_preds)
    del val_preds, val_targs, test_preds
    
    learn.model.close()
    del learn

In [ ]:
%%time
folds_to_train = [0,2,4,7,8,9]

ys = []
preds = []
test_preds = np.zeros((18000, 202, 202))
for fold in folds_to_train:
    y = np.load(f'/home/radek/db/salt/val_targs_{name}_fold{fold}.npy')
    val_preds = np.load(f'/home/radek/db/salt/val_preds_{name}_fold{fold}.npy')
    preds.append(val_preds)
    ys.append(y)
    test_pred = np.load(f'/home/radek/db/salt/test_preds_{name}_fold{fold}.npy')
    test_preds += test_pred / len(folds_to_train)

np.save(f'/home/radek/db/salt/val_preds_{name}.npy', np.concatenate(preds))
np.save(f'/home/radek/db/salt/val_targs_{name}.npy', np.concatenate(ys))
np.save(f'/home/radek/db/salt/test_preds_{name}.npy', test_preds)

In [ ]:
%%time
for fold in folds_to_train:
    val_preds = np.load(f'/home/radek/db/salt/val_preds_{name}_fold{fold}.npy')
    val_targs = np.load(f'/home/radek/db/salt/val_targs_{name}_fold{fold}.npy')
    print(f'Part {fold}: {accuracy_np(val_preds, val_targs)}, {iou_metric(val_targs, val_preds > 0.5)}, {best_preds_t(val_preds, val_targs)}')

In [ ]:
%%time
for fold in folds_to_train:
    val_preds = np.load(f'/home/radek/db/salt/val_preds_{name}_fold{fold}.npy')
    val_targs = np.load(f'/home/radek/db/salt/val_targs_{name}_fold{fold}.npy')
    print(f'Part {fold}: {accuracy_np(val_preds, val_targs)}, {iou_metric(val_targs, val_preds > 0.5)}, {best_preds_t(val_preds, val_targs)}')

In [ ]:
%%time
for fold in folds_to_train:
    val_preds = np.load(f'/home/radek/db/salt/val_preds_{name}_fold{fold}.npy')
    val_targs = np.load(f'/home/radek/db/salt/val_targs_{name}_fold{fold}.npy')
    print(f'Part {fold}: {accuracy_np(val_preds, val_targs)}, {iou_metric(val_targs, val_preds > 0.5)}, {best_preds_t(val_preds, val_targs)}')

In [ ]:
%%time
for fold in folds_to_train:
    val_preds = np.load(f'/home/radek/db/salt/val_preds_{name}_fold{fold}.npy')
    val_targs = np.load(f'/home/radek/db/salt/val_targs_{name}_fold{fold}.npy')
    print(f'Part {fold}: {accuracy_np(val_preds, val_targs)}, {iou_metric(val_targs, val_preds > 0.5)}, {best_preds_t(val_preds, val_targs)}')

In [ ]:
val_preds = np.load(f'/home/radek/db/salt/val_preds_{name}.npy')
val_targs = np.load(f'/home/radek/db/salt/val_targs_{name}.npy')

In [ ]:
iou_metric(val_targs, val_preds > 0.5)

In [ ]:
iou_metric(val_targs, val_preds > 0.5)

In [ ]:
iou_metric(val_targs, val_preds > 0.5)

In [ ]:
iou_metric(val_targs, val_preds > 0.5)

In [ ]:
%run diagnostics.ipynb

In [ ]:
%run diagnostics.ipynb

In [ ]:
test_preds = np.load(f'/home/radek/db/salt/test_preds_{name}.npy')

In [ ]:
# db = get_data_bunch(sz=202, bs=16, part=0, trn_tfms=trn_tfms)

In [ ]:
preds_to_sub(test_preds, db.test_dl.dl.dataset.x, 0.5, 0, name)

In [ ]:
!kaggle competitions submit -c tgs-salt-identification-challenge -f ../subs/{name}.csv.gz  -m {name}